### ライブラリのインポート

In [45]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os
import math
import optuna
import re

### 計算用関数

In [46]:
# 行数を指定すると所用時間を返す関数
def knapsack(row_x, data_x,required_time_x, waiting_time_x, move_time_x, attraction_no_x):
    waiting_time_x = data_x.iloc[row_x, attraction_no_x]
    ans = required_time_x + waiting_time_x + move_time_x
    return ans


# 合計所要時間と合計人気度を算出する関数
def ride_calculation(data, ride_lst, required_times, popularity, move_time):
    # print(data)
    # print(ride_lst)
    # print(required_times)
    # print(move_time)
    total_popularity = 0
    total_time = 0
    ride_time = []
    new_ride_lst = []
    # start_time = data.index[0]
    # print("start time is:", calculate_time(start_time))
    row = 0
    try:
        for i in ride_lst:
            # total_timeが675以上かつiが0または1の場合にループを強制終了
            # ソアリン、トイストーリーMは20:15以降DPA所持者のみ乗車可能
            if (data.index[row]) >= 660 and (i == 0 or i == 1):
                continue
            
            else:
                # iが8以上の場合、次のループへ(該当しないアトラクション)
                if i >= 8 or i < 0:
                    # print("不明なアトラクション：", i)
                    continue
                else:
                    # print("-" * 40)
                    # print(calculate_time(data.index[row]))
                    ride_time.append(data.index[row])
                    # print(attractions.get(i))
                    # print("移動時間", move_time)
                    waiting_time = data.iloc[row, i]
                    # print("待ち時間:", waiting_time)
                    required_time = required_times[i]
                    # print("観賞時間:", required_time)
                    ans = knapsack(row, data, required_time, waiting_time, move_time, i)
                    # print("合計所要時間:", ans)
                    total_time += ans
                    row = int(np.ceil(total_time / 15))
                    total_popularity += popularity[i]
                    new_ride_lst.append(i)
    except Exception as e:
        # print("全てのアトラクションを乗ることができません")
        pass

    # print("-" * 40)
    # print(total_popularity)
    return total_popularity, total_time, new_ride_lst, ride_time

In [47]:
#initial value.
lamb=1.0
tt_best=720
j=0
NN=10
#while((j<15)& (tt_best>660)):
#while((j<15)& (np.abs(tt_best-500)>160)):
while(np.abs(tt_best-600)>60):
#for j in range(2):
    print(tt_best)
    total_time_list=[]
    def objective(trial):
        # アトラクションの所要時間と人気度
        required_times = [5, 7, 2, 3, 3, 2, 23, 30]  # 所要時間
        popularity = [84, 69, 55, 45, 33, 28, 19, 22]  # 人気度
        move_time = 15

        file_path = '0724/2022-07-24.csv'
        # print(file_path)
        data = pd.read_csv(file_path)
        data = data.set_index('時間')

        # 乗り物の数を決定
        n_rides = trial.suggest_int('n_rides', 1, 20)  # 最大15の乗り物を仮定

        # 乗り物の定義
        ride_lst = [trial.suggest_int(f'ride_{i}', 0, 7) for i in range(n_rides)]

        # ride_calculation関数の呼び出し
        total_popularity, total_time, new_ride_lst, ride_time = ride_calculation(data, ride_lst, required_times, popularity, move_time)

        # 制約条件の範囲
        max_allowed_time = 660  # パーク内にいる時間 9:00~20:00 計660分 (20時以降はファストパスのみ受け入れと仮定)
        #trial.set_user_attr("constraints", [1000*(max_allowed_time - total_time)])
        print("total_time:",total_time)
        print("total_popularity:",total_popularity)
        total_time_list.append(total_time)
        return total_popularity+lamb*(max_allowed_time - total_time)
    
    # Optunaによる最適化
    study = optuna.create_study(direction="maximize")  # 人気度を最大化
    study.optimize(objective, n_trials = NN)
    best_trial = study.best_trial
    tt_best = total_time_list[best_trial.number] #最適解における所要時間の値。これをペナルティに用いる。
    
    best_trial = study.best_trial
    #print(f"Best trial: {best_trial.number}")
    #print(f"Best popularity: {best_trial.value}")
    #print(f"Parameters: {best_trial.params}")
    #print("total_time here is:", tt_best)
    
    
    lamb_list=[]
    def objective_dual(trial_dual):
        #lamb_list=[]
        # アトラクションの所要時間と人気度
        required_times = [5, 7, 2, 3, 3, 2, 23, 30]  # 所要時間
        popularity = [84, 69, 55, 45, 33, 28, 19, 22]  # 人気度
        move_time = 15

        file_path = '0724/2022-07-24.csv'
        # print(file_path)
        data = pd.read_csv(file_path)
        data = data.set_index('時間')

        # 乗り物の数を決定
        n_rides = trial_dual.suggest_int('n_rides', 1, 20)  # 最大15の乗り物を仮定

        # 乗り物の定義
        ride_lst = [trial_dual.suggest_int(f'ride_{i}', 0, 7) for i in range(n_rides)]

        # ride_calculation関数の呼び出し
        total_popularity, total_time, new_ride_lst, ride_time = ride_calculation(data, ride_lst, required_times, popularity, move_time)

        # 制約条件の範囲
        max_allowed_time = 660  # パーク内にいる時間 9:00~20:00 計660分 (20時以降はファストパスのみ受け入れと仮定)
        #lamb_x = 0.8*trial_dual.suggest_int('n_rides', 1, 3)
        lamb_x = trial_dual.suggest_float('lamb_x', 0.5, 2)
        #print("lamb:",lamb_x)
        lamb_list.append(lamb_x)
        #print("total_time:",total_time)
        #print("total_popularity:",total_popularity)
        penalty=max(0,(max_allowed_time - tt_best))
        #print("penalty",penalty)
        return total_popularity+lamb_x*penalty
    
    
    study_dual = optuna.create_study(direction="minimize")  # 人気度を最小化
    study_dual.optimize(objective_dual, n_trials = NN)
    
    
    best_trial_dual = study_dual.best_trial
    lamb = lamb_list[best_trial_dual.number]
    j=j+1
    print("tt_best2:",tt_best)
    #print("lamb is",lamb)

[I 2024-10-14 12:11:52,914] A new study created in memory with name: no-name-48a7c546-7838-45f1-bb84-85ff0e55d913
[I 2024-10-14 12:11:52,924] Trial 0 finished with value: 600.0 and parameters: {'n_rides': 6, 'ride_0': 7, 'ride_1': 6, 'ride_2': 3, 'ride_3': 5, 'ride_4': 5, 'ride_5': 2}. Best is trial 0 with value: 600.0.
[I 2024-10-14 12:11:52,924] Trial 1 finished with value: 635.0 and parameters: {'n_rides': 13, 'ride_0': 3, 'ride_1': 3, 'ride_2': 3, 'ride_3': 1, 'ride_4': 3, 'ride_5': 3, 'ride_6': 3, 'ride_7': 5, 'ride_8': 6, 'ride_9': 1, 'ride_10': 5, 'ride_11': 0, 'ride_12': 1}. Best is trial 1 with value: 635.0.
[I 2024-10-14 12:11:52,936] Trial 2 finished with value: 568.0 and parameters: {'n_rides': 9, 'ride_0': 5, 'ride_1': 6, 'ride_2': 7, 'ride_3': 6, 'ride_4': 5, 'ride_5': 0, 'ride_6': 0, 'ride_7': 3, 'ride_8': 1}. Best is trial 1 with value: 635.0.
[I 2024-10-14 12:11:52,944] Trial 3 finished with value: 625.0 and parameters: {'n_rides': 19, 'ride_0': 3, 'ride_1': 4, 'ride_2

720
total_time: 257
total_popularity: 197
total_time: 661
total_popularity: 636
total_time: 490
total_popularity: 398
total_time: 718
total_popularity: 683
total_time: 593
total_popularity: 402
total_time: 336
total_popularity: 228
total_time: 733
total_popularity: 665
total_time: 158
total_popularity: 129
total_time: 711
total_popularity: 628
total_time: 738
total_popularity: 522
tt_best2: 661
661
total_time: 598
total_popularity: 502
total_time: 341
total_popularity: 192
total_time: 748
total_popularity: 496
total_time: 738
total_popularity: 461
total_time: 648
total_popularity: 541
total_time: 511
total_popularity: 442
total_time: 83
total_popularity: 41
total_time: 614
total_popularity: 526
total_time: 417
total_popularity: 374


[I 2024-10-14 12:11:53,105] Trial 9 finished with value: 503.8358271338704 and parameters: {'n_rides': 14, 'ride_0': 7, 'ride_1': 0, 'ride_2': 1, 'ride_3': 5, 'ride_4': 1, 'ride_5': 0, 'ride_6': 1, 'ride_7': 6, 'ride_8': 5, 'ride_9': 6, 'ride_10': 4, 'ride_11': 4, 'ride_12': 7, 'ride_13': 3}. Best is trial 8 with value: 611.206415668435.
[I 2024-10-14 12:11:53,114] A new study created in memory with name: no-name-98eea886-8f55-4aec-a12a-742b9f00aaeb
[I 2024-10-14 12:11:53,114] Trial 0 finished with value: 301.9651729429898 and parameters: {'n_rides': 1, 'ride_0': 2, 'lamb_x': 1.01631758412753}. Best is trial 0 with value: 301.9651729429898.
[I 2024-10-14 12:11:53,124] Trial 1 finished with value: 740.9359961161558 and parameters: {'n_rides': 12, 'ride_0': 7, 'ride_1': 7, 'ride_2': 7, 'ride_3': 6, 'ride_4': 5, 'ride_5': 4, 'ride_6': 5, 'ride_7': 5, 'ride_8': 5, 'ride_9': 3, 'ride_10': 3, 'ride_11': 6, 'lamb_x': 1.6540575971858262}. Best is trial 0 with value: 301.9651729429898.
[I 2024-

total_time: 737
total_popularity: 579
tt_best2: 417
417
total_time: 535
total_popularity: 441
total_time: 634
total_popularity: 512
total_time: 709
total_popularity: 590
total_time: 380
total_popularity: 206
total_time: 109
total_popularity: 85
total_time: 279
total_popularity: 219
total_time: 723
total_popularity: 460
total_time: 158
total_popularity: 129
total_time: 536
total_popularity: 455
total_time: 721
total_popularity: 568
tt_best2: 709
709
total_time: 117
total_popularity: 55
total_time: 493
total_popularity: 316


[I 2024-10-14 12:11:53,293] Trial 2 finished with value: 844.0761865253821 and parameters: {'n_rides': 8, 'ride_0': 7, 'ride_1': 6, 'ride_2': 0, 'ride_3': 4, 'ride_4': 0, 'ride_5': 4, 'ride_6': 5, 'ride_7': 1}. Best is trial 0 with value: 1114.2453276168699.
[I 2024-10-14 12:11:53,305] Trial 3 finished with value: 353.6114136948543 and parameters: {'n_rides': 17, 'ride_0': 2, 'ride_1': 0, 'ride_2': 5, 'ride_3': 6, 'ride_4': 5, 'ride_5': 1, 'ride_6': 4, 'ride_7': 6, 'ride_8': 3, 'ride_9': 7, 'ride_10': 7, 'ride_11': 4, 'ride_12': 0, 'ride_13': 1, 'ride_14': 4, 'ride_15': 2, 'ride_16': 3}. Best is trial 0 with value: 1114.2453276168699.
[I 2024-10-14 12:11:53,312] Trial 4 finished with value: 561.7917334801576 and parameters: {'n_rides': 10, 'ride_0': 6, 'ride_1': 1, 'ride_2': 6, 'ride_3': 6, 'ride_4': 5, 'ride_5': 7, 'ride_6': 4, 'ride_7': 6, 'ride_8': 4, 'ride_9': 2}. Best is trial 0 with value: 1114.2453276168699.
[I 2024-10-14 12:11:53,315] Trial 5 finished with value: 1123.082950877

...

tt_best2: 717
717
total_time: 38
total_popularity: 19
total_time: 447
total_popularity: 300
total_time: 708
total_popularity: 541
total_time: 720
total_popularity: 609
total_time: 632
total_popularity: 392
total_time: 190
total_popularity: 175
total_time: 714
total_popularity: 582
total_time: 355
total_popularity: 212
total_time: 733
total_popularity: 642
total_time: 627
total_popularity: 486
tt_best2: 733
733
total_time: 608
total_popularity: 435
total_time: 293
total_popularity: 246
total_time: 750
total_popularity: 572
total_time: 710
total_popularity: 578
total_time: 528
total_popularity: 365
total_time: 648
total_popularity: 423
total_time: 268
total_popularity: 201
total_time: 592
total_popularity: 481


[I 2024-10-14 12:11:56,544] Trial 8 finished with value: 994.1791404980854 and parameters: {'n_rides': 2, 'ride_0': 6, 'ride_1': 6}. Best is trial 8 with value: 994.1791404980854.
[I 2024-10-14 12:11:56,554] Trial 9 finished with value: 534.5223925622606 and parameters: {'n_rides': 9, 'ride_0': 2, 'ride_1': 4, 'ride_2': 7, 'ride_3': 2, 'ride_4': 3, 'ride_5': 1, 'ride_6': 3, 'ride_7': 1, 'ride_8': 7}. Best is trial 8 with value: 994.1791404980854.
[I 2024-10-14 12:11:56,554] A new study created in memory with name: no-name-20f81931-37e7-4ca6-9786-94e6bbf32c39
[I 2024-10-14 12:11:56,564] Trial 0 finished with value: 1565.3980636937608 and parameters: {'n_rides': 18, 'ride_0': 6, 'ride_1': 1, 'ride_2': 7, 'ride_3': 4, 'ride_4': 3, 'ride_5': 4, 'ride_6': 6, 'ride_7': 2, 'ride_8': 5, 'ride_9': 6, 'ride_10': 4, 'ride_11': 4, 'ride_12': 5, 'ride_13': 1, 'ride_14': 0, 'ride_15': 0, 'ride_16': 3, 'ride_17': 5, 'lamb_x': 1.8089007939961657}. Best is trial 0 with value: 1565.3980636937608.
[I 202

total_time: 76
total_popularity: 38
total_time: 587
total_popularity: 415
tt_best2: 76
76
total_time: 691
total_popularity: 645
total_time: 706
total_popularity: 526
total_time: 709
total_popularity: 627
total_time: 525
total_popularity: 412
total_time: 590
total_popularity: 552
total_time: 651
total_popularity: 643
total_time: 626
total_popularity: 495
total_time: 650
total_popularity: 549
total_time: 313
total_popularity: 291
total_time: 290
total_popularity: 237


[I 2024-10-14 12:11:56,739] Trial 7 finished with value: 612.2927009220184 and parameters: {'n_rides': 18, 'ride_0': 3, 'ride_1': 0, 'ride_2': 3, 'ride_3': 0, 'ride_4': 6, 'ride_5': 1, 'ride_6': 1, 'ride_7': 4, 'ride_8': 1, 'ride_9': 6, 'ride_10': 3, 'ride_11': 1, 'ride_12': 7, 'ride_13': 3, 'ride_14': 4, 'ride_15': 0, 'ride_16': 5, 'ride_17': 3, 'lamb_x': 1.0325223246687136}. Best is trial 5 with value: 123.40362035548758.
[I 2024-10-14 12:11:56,744] Trial 8 finished with value: 318.48054669782 and parameters: {'n_rides': 7, 'ride_0': 4, 'ride_1': 6, 'ride_2': 6, 'ride_3': 1, 'ride_4': 4, 'ride_5': 1, 'ride_6': 1, 'lamb_x': 0.8311718553133309}. Best is trial 5 with value: 123.40362035548758.
[I 2024-10-14 12:11:56,751] Trial 9 finished with value: 664.249440194043 and parameters: {'n_rides': 17, 'ride_0': 4, 'ride_1': 5, 'ride_2': 3, 'ride_3': 1, 'ride_4': 5, 'ride_5': 2, 'ride_6': 7, 'ride_7': 3, 'ride_8': 0, 'ride_9': 2, 'ride_10': 3, 'ride_11': 7, 'ride_12': 7, 'ride_13': 3, 'ride_

tt_best2: 651


In [48]:
print(f"Best trial: {best_trial.number}")
print(f"Best popularity: {best_trial.value}")
print(f"Parameters: {best_trial.params}")
print("total_time here is:", tt_best)
#print("lamb is",lamb)

Best trial: 5
Best popularity: 648.2815115939452
Parameters: {'n_rides': 9, 'ride_0': 3, 'ride_1': 1, 'ride_2': 1, 'ride_3': 0, 'ride_4': 2, 'ride_5': 0, 'ride_6': 0, 'ride_7': 1, 'ride_8': 0}
total_time here is: 651
